In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split

In [ ]:
pd.set_option("display.max_columns",None)

We tak small sample size

In [ ]:
train_data = pd.read_csv('/kaggle/input/amex-default-prediction/train_data.csv', nrows=200000)

In [ ]:
train_labels = pd.read_csv("/kaggle/input/amex-default-prediction/train_labels.csv", nrows=200000)

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/amex-default-prediction/sample_submission.csv")

In [ ]:
#select_cols = ['customer_ID',  'S_2', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
select_cols = ['customer_ID',   'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [ ]:
#test_data = pd.read_csv("/kaggle/input/amex-default-prediction/test_data.csv", usecols=select_cols)
#test_data = pd.read_parquet("/kaggle/input/amex-parquet/test_data.parquet", columns=select_cols)

In [ ]:
#test_data.head(2)

In [ ]:
#test_data.shape

In [ ]:
#sample_submission.shape

In [ ]:
print("Number of rows in train data {}".format(train_data.shape[0]))
print("Number of cols in train data {}".format(train_data.shape[1]))

In [ ]:
print("Number of rows in labels data {}".format(train_labels.shape[0]))
print("Number of cols in labels data {}".format(train_labels.shape[1]))

In [ ]:
train_labels.head(2)

find a unique customer id

In [ ]:
len(train_labels['customer_ID'].unique())

In [ ]:
#sample_submission.head()

Number of unique customer in train data

In [ ]:
train_data['customer_ID'].nunique() 

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
train_data = train_data[select_cols]

In [ ]:
print("Number of rows in train data {}".format(train_data.shape[0]))
print("Number of cols in train data {}".format(train_data.shape[1]))

In [ ]:
train_data.dtypes

In [ ]:
train_data = train_data.groupby(['customer_ID']).mean()

In [ ]:
train_data.shape

In [ ]:
train_data.head(2)

In [ ]:
train_data.reset_index()

In [ ]:
feature_label = train_data.merge(train_labels, left_on='customer_ID', right_on='customer_ID')

In [ ]:
feature_label.head()

In [ ]:
feature_label.shape

In [ ]:
tmp = feature_label['target'].value_counts().div(len(feature_label)).mul(100)

In [ ]:
ax = sns.barplot(x= tmp.index, y= tmp.values)
ax.bar_label(ax.containers[0], fmt="%.f%%")
plt.title("Distribution of a target variable")
plt.ylabel("Percentage [%]")
plt.show()

In [ ]:
feature_cols = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126',  'D_66', 'D_68']
target_cols = ['target']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(feature_label[feature_cols],feature_label[target_cols] , test_size = 0.2, random_state=90)

In [ ]:
model = Sequential()
model.add(Dense(24, input_dim=9, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
print(model.summary())

In [ ]:
from tensorflow.keras.optimizers import SGD 

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history  = model.fit(x_train, y_train, epochs=10, batch_size=20)

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(x_train,y_train)
print('Accuracy: %.2f'%(accuracy*100))

In [ ]:
#saved performance result
history_dict = history.history

loss_values = history_dict['loss']
#val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values)+1)


#line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label = 'Training Loss')
#plt.setp(line1, linewidth=2.0, marker='+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker='4', markersize=10.0)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
test_prediction = model.predict(x_test)
# round predictions
test_rounded = [round(x[0]) for x in test_prediction]

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, test_rounded))